# Lab SQL 9
In this lab we will find the customers who were active in consecutive months of May and June. Follow the steps to complete the analysis.


## Create a table `rentals_may` to store the data from rental table with information for the month of May.

In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass 
password = getpass.getpass()

········


In [2]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [11]:
rental = pd.read_sql_query('SELECT * FROM rental', engine)
rental.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [22]:
query = '''CREATE TABLE rentals_may (
rental_id int(10) UNIQUE NOT NULL,
rental_date DATETIME DEFAULT NULL,
inventory_id int(10) DEFAULT NULL,
customer_id int(10) DEFAULT NULL,
return_date DATETIME DEFAULT NULL,
staff_id int(10) DEFAULT NULL,
last_update DATETIME DEFAULT NULL,
CONSTRAINT PRIMARY KEY (rental_id)
)
'''

rentals_may = pd.read_sql_query(query, engine)
rentals_may.head()

# this gave me an error first = ResourceClosedError: This result object does not return rows. It has been closed automatically.
#then second error says the table is already created

OperationalError: (pymysql.err.OperationalError) (1050, "Table 'rentals_may' already exists")
[SQL: CREATE TABLE rentals_may (
rental_id int(10) UNIQUE NOT NULL,
rental_date DATETIME DEFAULT NULL,
inventory_id int(10) DEFAULT NULL,
customer_id int(10) DEFAULT NULL,
return_date DATETIME DEFAULT NULL,
staff_id int(10) DEFAULT NULL,
last_update DATETIME DEFAULT NULL,
CONSTRAINT PRIMARY KEY (rental_id)
)
]
(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [24]:
rentals_may = pd.read_sql_query('SELECT * FROM rentals_may', engine)
rentals_may.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update


## Insert values in the table `rentals_may` using the table rental, filtering values only for the month of May.
 

In [32]:
query = '''INSERT INTO rentals_may
SELECT * FROM rental
WHERE rental_date LIKE '2005-05%'
'''

rentals_may = pd.read_sql_query(query, engine)
rentals_may.head()

IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '1' for key 'rentals_may.PRIMARY'")
[SQL: INSERT INTO rentals_may
SELECT * FROM rental
WHERE rental_date LIKE '2005-05%'
]
(Background on this error at: http://sqlalche.me/e/13/gkpj)

In [33]:
rentals_may = pd.read_sql_query('SELECT * FROM rentals_may', engine)
rentals_may.head()

#why is the error if data is inserted?

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


## Create a table `rentals_june` to store the data from rental table with information for the month of June.


In [30]:
query = '''CREATE TABLE rentals_june (
rental_id int(10) UNIQUE NOT NULL,
rental_date DATETIME DEFAULT NULL,
inventory_id int(10) DEFAULT NULL,
customer_id int(10) DEFAULT NULL,
return_date DATETIME DEFAULT NULL,
staff_id int(10) DEFAULT NULL,
last_update DATETIME DEFAULT NULL,
CONSTRAINT PRIMARY KEY (rental_id)
)
'''

rentals_june = pd.read_sql_query(query, engine)

ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [34]:
rentals_june = pd.read_sql_query('SELECT * FROM rentals_june', engine)
rentals_june.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update


## Insert values in the table `rentals_june` using the table rental, filtering values only for the month of June.
 

In [35]:
query = '''INSERT INTO rentals_june
SELECT * FROM rental
WHERE rental_date LIKE '2005-06%'
'''

rentals_june = pd.read_sql_query(query, engine)
rentals_june.head()

ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [38]:
rentals_june = pd.read_sql_query('SELECT * FROM rentals_june', engine)
rentals_june.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1158,2005-06-14 22:53:33,1632,416,2005-06-18 21:37:33,2,2006-02-15 21:30:53
1,1159,2005-06-14 22:55:13,4395,516,2005-06-17 02:11:13,1,2006-02-15 21:30:53
2,1160,2005-06-14 23:00:34,2795,239,2005-06-18 01:58:34,2,2006-02-15 21:30:53
3,1161,2005-06-14 23:07:08,1690,285,2005-06-21 17:12:08,1,2006-02-15 21:30:53
4,1162,2005-06-14 23:09:38,987,310,2005-06-23 22:00:38,1,2006-02-15 21:30:53


## Check the number of rentals for each customer for May.


In [44]:
query = '''SELECT customer_id, count(rental_id) as n_rentals
FROM rentals_may
GROUP BY customer_id
ORDER BY n_rentals DESC'''
n_rentals_may = pd.read_sql_query(query, engine)
n_rentals_may

,customer_id,n_rentals
0,197,8
1,506,7
2,109,7
3,269,6
4,239,6
...,...,...
515,431,1
516,351,1
517,10,1
518,136,1


## Check the number of rentals for each customer for June.
 

In [45]:
query = '''SELECT customer_id, count(rental_id) as n_rentals
FROM rentals_june
GROUP BY customer_id
ORDER BY n_rentals DESC'''
n_rentals_june = pd.read_sql_query(query, engine)
n_rentals_june

,customer_id,n_rentals
0,31,11
1,454,10
2,329,9
3,295,9
4,561,9
...,...,...
585,425,1
586,412,1
587,335,1
588,22,1


## Create a Python connection with SQL database and retrieve the results of the last two queries (also mentioned below) as dataframes:

  - Check the number of rentals for each customer for May
  - Check the number of rentals for each customer for June

    **Hint**: You can store the results from the two queries in two separate dataframes.
 

In [ ]:
# not sure what do do here but I think I did this in the two previous questions.
#Then I think I dont know how to do the two previous questions :( 

## Write a function that checks if customer borrowed more or less films in the month of June as compared to May.
 
 **Hint**: For this part, you can create a join between the two dataframes created before, using the merge function available for pandas dataframes. Here is a link to the documentation for the [merge function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html).